# Análisis de datos de Televisión Digital Terrestre (TDT)
## Unidad de Medios y Contenidos Audiovisuales
### Subdirección de validación

<img src="./assets/IFTheader.png"
     alt="IFTicon"
     style = 'position:absolute; top:0; right:0; padding:2px;'/>
     

La presente *notebook* tiene por objetivo ser una herramienta de análisis y trabajo, que de la mano con las bases de datos, faciliten y optimicen los procesos realizados dentro de la subidirección. Especificamente aquellos que implican el procesamiento de coberturas, listado de localidades, valores porcentuales de cobertura y extensión geográfica; por ejemplo, las publicaciones del análisis para la **Retransmisión de señales**, cuyo sustento es el análisis de cobertura porcentual nacional por cada canal de programación transmitido por las estaciones de TDT.

## Iniciación y preparación de insumos

In [8]:
#Bibliotecas generales
import glob
import os

#Bibliotecas pata análisis de datos
import csv
import pandas as pd
import collections
import geopandas as gpd
import numpy as np
import mapclassify as mc

#Bibliotecas de visualización de datos
import seaborn as sns
import matplotlib as mpl
from tqdm.notebook import tqdm, trange

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure


In [5]:
#Definimos la ruta del archivo base de localidades,con la cual se compararan las coberturas.

inegi_path='./data/Localidades_20100.csv'
inegi_data = pd.read_csv(inegi_path,header=0, delimiter = ";", encoding = 'latin-1')

distintivos= []
localidades=[]
nom_loc = []
history = []

In [6]:
inegi_data

,_ID,NO_ENT,ENTIDAD,NO_MPIO,MUNICIPIO,NO_LOC,Location,GM_2010,LONG,LAT,...,MUJERES,TOTAL_HOGA,VIVPARHA,O_VIVPAR,VPH_RADIO,VPH_TV,POB_0_17,POB_18_59,POB_MAYOR_,OID
0,10010001,1,Aguascalientes,1,Aguascalientes,1,AGUASCALIENTES,Muy bajo,1021746,215251,...,373528.0,184123.0,185050.0,718066.0,167880.0,181062.0,255352.0,410259.0,56639.0,1
1,10010094,1,Aguascalientes,1,Aguascalientes,94,GRANJA ADELITA,x,1022225,215219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,10010096,1,Aguascalientes,1,Aguascalientes,96,AGUA AZUL,Alto,1022127,215303,...,16.0,11.0,11.0,37.0,8.0,11.0,12.0,21.0,4.0,3
3,10010100,1,Aguascalientes,1,Aguascalientes,100,RANCHO ALEGRE,x,1022222,215116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,10010102,1,Aguascalientes,1,Aguascalientes,102,LOS ARBOLITOS [RANCHO],x,1022126,214649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192240,320580036,32,Zacatecas,58,Santa María de la Paz,36,MARINES,x,1032337,213347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192241
192241,320580037,32,Zacatecas,58,Santa María de la Paz,37,MESA GRANDE,Medio,1032457,213317,...,99.0,47.0,47.0,192.0,43.0,43.0,71.0,38.0,83.0,192242
192242,320580039,32,Zacatecas,58,Santa María de la Paz,39,SAN ISIDRO,x,1032014,212916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192243
192243,320580041,32,Zacatecas,58,Santa María de la Paz,41,SAN MIGUEL TEPETITLÁN,Medio,1032007,213014,...,34.0,29.0,29.0,75.0,24.0,25.0,14.0,26.0,35.0,192244


In [15]:
PATH_DB = './data/BD_Canales_MCMO.xlsx'

data_cp=pd.read_excel(PATH_DB, sheet_name=0,header=2)


In [23]:
prog_channel_counter = collections.Counter(data_cp['PROGRAMACIÓN'])

In [24]:
prog_channel_counter

Counter({'4 XHS': 1,
         'EL CANAL DE LAS NOTICIAS': 3,
         'XHBC CANAL 4': 1,
         'CANAL 45': 1,
         'CANAL 5': 98,
         'NU9VE': 46,
         'XEWT CANAL 12': 1,
         'TELSUSA': 16,
         'CANAL 28 DE CHIHUAHUA': 1,
         'LAS ESTRELLAS': 129,
         '44 ALTERNATIVO': 1,
         'CANAL CATORCE': 27,
         'WORLD TV': 1,
         'CANAL 28.1': 1,
         'CANAL 28.2': 1,
         'MULTIMEDIOS': 5,
         'CANAL 13 DEL PARRAL': 1,
         'SUPER CHANNEL 12.1': 1,
         'SUPER CHANNEL 12.2': 1,
         'CANAL 5 SABINAS': 1,
         'CANAL 12': 1,
         'HERALDO TV': 1,
         'FORO TV': 36,
         'CV SHOPPING': 15,
         'CANAL 22.1': 1,
         'ADN 40': 71,
         'CANAL 12 DURANGO': 1,
         'CANAL 10 ': 1,
         'EXPRESA TV': 1,
         'TV4': 27,
         'EL CANAL DE SAN MIGUEL': 1,
         'CANAL 8 GUADALAJARA': 1,
         'C7NOTICIAS.1': 3,
         'CANAL 4': 1,
         'MÁSVISIÓN': 1,
         'CANAL 44':

## Análisis inicial  de TDT

## Análisis de MCMO

## Resultados y valor agregado TDT INEGI 2020

## Summary